In [ ]:


import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import torch
import os
from scipy.stats import linregress
import ast
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

matplotlib.rcParams['font.family'] = 'Times New Roman'
matplotlib.rcParams['font.size'] = 12
matplotlib.rcParams['axes.linewidth'] = 1
matplotlib.rcParams['xtick.direction'] = 'in'
matplotlib.rcParams['ytick.direction'] = 'in'
matplotlib.rcParams['xtick.major.width'] = 1
matplotlib.rcParams['ytick.major.width'] = 1
matplotlib.rcParams['legend.frameon'] = False
try:
    folder = os.path.dirname(os.path.abspath(__file__))
except NameError:
    folder = os.getcwd()

glist = [1.1, 1.3, 1.5]
step_t = 2000
delta_t = 0.1
tail_range = 200
N1 = 2000
N2 = 6000

# =================================================
filename = f"DMFT_kinetic_vs_g{glist[0]:.2f}to{glist[-1]:.2f}.csv"
df_dmft = pd.read_csv(os.path.join(folder, filename))
df_Slope = (df_dmft['sqrt_Gamma0_mean'])
df_Slope_std =(df_dmft['sqrt_Gamma0_std'])  


Simu1 = pd.read_csv(f"RNNSIM_N{N1}.csv")
Arc_RNN_Mean1 = Simu1["Arc_RNN_Mean"].apply(ast.literal_eval).tolist()
Arc_RNN_Std1  = Simu1["Arc_RNN_Std"].apply(ast.literal_eval).tolist()
Slope_RNN1 = Simu1["Slope_List_RNN"].values
Intercept_RNN1 = Simu1["Intercept_List_RNN"].values
Stderr_RNN1 = Simu1["Stderr_List_RNN"].values
Simu2 = pd.read_csv(f"RNNSIM_N{N2}.csv")
Arc_RNN_Mean2 = Simu2["Arc_RNN_Mean"].apply(ast.literal_eval).tolist()
Arc_RNN_Std2  = Simu2["Arc_RNN_Std"].apply(ast.literal_eval).tolist()
Slope_RNN2 = Simu2["Slope_List_RNN"].values
Intercept_RNN2 = Simu2["Intercept_List_RNN"].values
Stderr_RNN2 = Simu2["Stderr_List_RNN"].values

# =================================================
Marker_List = ['o', 's', '^']
Color_List = ['#D81B60', '#2E75B6', '#7D3C98' ]
Color_Shade = ['#F8BBD0', '#AED6F1', '#D7BDE2' ]
Color_List1 = ['#F06292','#5DADE2', '#AF7AC5']

Color_List = ['pink', 'skyblue', '#C896C8']
Color_Shade = ['#FFD1DC', '#D0ECFF', '#E8D2EB']
Color_List1 = ['#D95C8D', '#4B9CD3', '#9A4EA9']

fig, axs = plt.subplots(2, 2, figsize=(8, 8), dpi=300)
t_mark = 180.0
tail_plot = 20
time_axis = np.arange(step_t + 1) * delta_t

# ====================================================
for i, g in enumerate(glist):
    axs[0,0].plot(time_axis, Arc_RNN_Mean1[i], '--', lw=2, color=Color_List[i], label=rf'$g={g:.1f}$')
    axs[0,0].fill_between(time_axis, np.array(Arc_RNN_Mean1[i])-np.array(Arc_RNN_Std1[i]), np.array(Arc_RNN_Mean1[i])+np.array(Arc_RNN_Std1[i]), color=Color_Shade[i], alpha=0.8)

axs[0,0].axvline(t_mark, color='k', ls='--', lw=1)
t_idx = int(t_mark / delta_t) 
y_points = [Arc_RNN_Mean1[i][t_idx] for i in range(3)]
for i, y in enumerate(y_points):
    axs[0,0].scatter(t_mark, y, s=40, color=Color_List[i],
                   edgecolor='k', zorder=5, marker=Marker_List[i])
axs[0,0].set_ylabel(rf'$\langle s (t) \rangle $,$N={N1}$', fontsize=14)
axs[0,0].set_xlim(0, 200)
axs[0,0].set_ylim(0, 33)
axs[0,0].set_title("(a)", loc='center', fontsize=16)
axs[0,0].legend(fontsize=9, loc='upper left')
axs[0,0].tick_params(axis='both', labelsize=12)

# ======================================================
x_fit = np.arange(tail_range) * delta_t
for i, _ in enumerate(glist):
    axs[0,1].plot(x_fit, Slope_RNN1[i] * x_fit + y_points[0], '--', color=Color_List[i], lw=2)
    axs[0,1].fill_between(x_fit,
                        (Slope_RNN1[i] - Stderr_RNN1[i]) * x_fit + y_points[0],
                        (Slope_RNN1[i] + Stderr_RNN1[i]) * x_fit + y_points[0],
                        color=Color_Shade[i], alpha=0.8,
                        label=rf'$k_s={Slope_RNN1[i]:.3f}\pm{Stderr_RNN1[i]:.3f}$')
    axs[0,1].plot(x_fit, df_Slope[i] * x_fit + y_points[0], '-', color=Color_List1[i], alpha=0.9,lw=2)
    axs[0,1].fill_between(x_fit,
                        (df_Slope[i] - df_Slope_std[i]) * x_fit + y_points[0],
                        (df_Slope[i] + df_Slope_std[i]) * x_fit + y_points[0],
                        color=Color_List1[i], alpha=0.9 ,edgecolor='none',
                        label=rf'$k_d={df_Slope[i]:.3f}\pm{df_Slope_std[i]:.3f}$')  
    
axs[0,1].set_xlim(0, x_fit[-1])
axs[0,1].set_ylim( y_points[0], y_points[0] + 3.5)
axs[0,1].legend(fontsize=9, loc='upper left')
axs[0,1].set_title("(b)", loc='center', fontsize=16)
x1ticks_vals = [0, 4, 8, 12, 16, 20]
xlabels = ['180','184', '188', '112', '116', '200']
axs[0,1].set_xticks(x1ticks_vals, xlabels)
axs[0,1].tick_params(axis='both', labelsize=12)

# ====================================================
for i, g in enumerate(glist):
    axs[1,0].plot(time_axis, Arc_RNN_Mean2[i], '--', lw=2, color=Color_List[i], label=rf'$g={g:.1f}$')
    axs[1,0].fill_between(time_axis, np.array(Arc_RNN_Mean2[i])-np.array(Arc_RNN_Std2[i]), np.array(Arc_RNN_Mean2[i])+np.array(Arc_RNN_Std2[i]), color=Color_Shade[i], alpha=0.8)

axs[1,0].axvline(t_mark, color='k', ls='--', lw=1)
t_idx = int(t_mark / delta_t) 
y_points = [Arc_RNN_Mean2[i][t_idx] for i in range(3)]
for i, y in enumerate(y_points):
    axs[1,0].scatter(t_mark, y, s=40, color=Color_List[i],
                   edgecolor='k', zorder=5, marker=Marker_List[i])
axs[1,0].set_xlabel(r'$t$', fontsize=14)
axs[1,0].set_ylabel(rf'$\langle s (t) \rangle $,$N={N2}$', fontsize=14)
axs[1,0].set_xlim(0, 200)
axs[1,0].set_ylim(0, 33)
axs[1,0].set_title("(c)", loc='center', fontsize=16)
axs[1,0].legend(fontsize=9, loc='upper left')
axs[1,0].tick_params(axis='both', labelsize=12)

# ======================================================
x_fit = np.arange(tail_range+1) * delta_t
for i, _ in enumerate(glist):
    axs[1,1].plot(x_fit, Slope_RNN2[i] * x_fit + y_points[0], '--', color=Color_List[i],lw=2)
    axs[1,1].fill_between(x_fit,
                        (Slope_RNN2[i] - Stderr_RNN2[i]) * x_fit + y_points[0],
                        (Slope_RNN2[i] + Stderr_RNN2[i]) * x_fit + y_points[0],
                        color=Color_Shade[i],alpha=0.8,
                        label=rf'$k_s={Slope_RNN2[i]:.3f}\pm{Stderr_RNN2[i]:.3f}$')
    axs[1,1].plot(x_fit, df_Slope[i] * x_fit + y_points[0], '-', color=Color_List1[i], alpha=0.9,lw=2)
    axs[1,1].fill_between(x_fit,
                        (df_Slope[i] - df_Slope_std[i]) * x_fit + y_points[0],
                        (df_Slope[i] + df_Slope_std[i]) * x_fit + y_points[0],
                        color=Color_List1[i], alpha=0.9, edgecolor='none',
                        label=rf'$k_d={df_Slope[i]:.3f}\pm{df_Slope_std[i]:.3f}$')  
    
axs[1,1].set_xlabel(r'$t$', fontsize=14)
# axs[1,1].set_ylabel(r'$\bar{s}(t)$', fontsize=14)
axs[1,1].set_xlim(0, x_fit[-1])
axs[1,1].set_ylim( y_points[0], y_points[0] + 3.3)
axs[1,1].legend(fontsize=9, loc='upper left')
axs[1,1].set_title("(d)", loc='center', fontsize=16)
x1ticks_vals = [0,4,8,12,16,20]
xlabels = ['180','184', '188', '112', '116', '200']
axs[1,1].set_xticks(x1ticks_vals, xlabels)
axs[1,1].tick_params(axis='both', labelsize=12)

plt.savefig("ARC.pdf", format="pdf")




